# Deep Dive into LangChain

In [1]:
# pip install -r ./requirements.txt -q

### Python-dotenv

In [5]:
import os 
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

'sk-proj-dsxvcH7KXpdycO17nObyT3BlbkFJivJ8OhSW43DNJlVp6w1D'

## Chat Models: GPT-3.5 Turbo and GPT-4

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
# output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', max_tokens=100, temperature=0.5, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=['\n', ')
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo')
print(output.content)

Quantum mechanics is a fundamental theory in physics that describes the behavior of particles at the smallest scales, including the peculiar phenomena of superposition, entanglement, and uncertainty, which defy classical intuition.


In [10]:
from langchain.schema import (
    SystemMessage, # corresponds to the OpenAI chat completion API
    AIMessage,     # Assistent message
    HumanMessage   # Human messages or prompt
)

messages = [    
    SystemMessage(content='You are a physicist and respond only in portuguese.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

A mecânica quântica é a teoria que descreve o comportamento das partículas subatômicas em termos de probabilidades e ondas de probabilidade.
